# Wie bekommt man die Daten?

## Benötigte Pakete laden

In [ ]:
import pandas as pd

## Einlesen der Metadaten

In [ ]:
data = pd.read_csv(
    "GEO Accession viewer.html",
    skiprows=21,
    skipfooter=339 - 309,
    sep="\t",
    engine="python",
)

In [ ]:
data

In [ ]:
data.columns = data.columns.str.replace(
    "<[^>]+>", "", regex=True
)  # Entfernen der Formatierungen

In [ ]:
data["relapse"] = data["relapse (1=True)"].replace(
    {1: "yes", 0: "no"}
)  # Formatierung der Relapse Information

In [ ]:
data["Brain relapses"] = data["Brain relapses (1=yes, 0=no)"].replace(
    {1: "yes", 0: "no"}
)  # Formatierung der Brain Relapse Information

In [ ]:
data

In [ ]:
data = data.drop(
    columns=[
        "relapse (1=True)",
        "Brain relapses (1=yes, 0=no)",
        "PID",
        "lymph node status",
    ]
)  # Entfernen von nicht benötigten Informationen

In [ ]:
data.to_csv("GSE2034_metadata.csv", index=False)  # Speichern der Metadaten

## Einlesen der Expressionsdaten

In [ ]:
data = pd.read_csv("GSE2034-22071.txt", sep="\t", skiprows=287, index_col=None)

In [ ]:
data  # ID_REF ist falsch eingerückt.

In [ ]:
data.columns = data.columns[1:].to_list() + [
    "to_be_deleted"
]  # Richtiges Einrücken der Spalten
data = data.dropna(
    axis=1, how="all"
)  # Entfernen von fehlenden Werten (Achtung, vorher schauen woher diese kommen!)

### ID's in Gennamen umwandeln

In [ ]:
mapping = pd.read_csv(
    "GPL96-57554.txt",
    comment="#",
    sep="\t",
    usecols=["ID", "Gene Symbol"],
    index_col="ID",
)

In [ ]:
mapping

In [ ]:
mapping = mapping.dropna()  # Entfernen von ID's ohne Gennamen

In [ ]:
mapping = (
    mapping.squeeze().str.split("///", expand=True)[0].str.strip()
)  # Auswahl von einem Genesymbol pro ID
mapping

In [ ]:
data.index = mapping.reindex(
    data.index
)  # Mapping von ID's in Daten auf Gennamen durch `mapping`

In [ ]:
data

In [ ]:
data = data[~data.index.isna()]  # Entfernen von ID's die nicht gemapped werden konnten

In [ ]:
data = (
    data.groupby(level=0).median()
)  # Manche Gene hatten mehrere ID's. Wir benutzen hier die Median-Genexpression

In [ ]:
data

In [ ]:
data.to_csv("GSE2034_data.csv")